<a href="https://colab.research.google.com/github/kurniacf/BigData_AssigmentWeek9_CollaborativeFiltering_BuildingRecomender/blob/master/Collaborative_Filtering_5025201073_Kurnia_Cahya_Febryanto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BIG DATA ASSIGNMENT WEEK 09
## Filtering Collaborative
- Name: Kurnia Cahya Febryanto
- Student ID: 5025201073
- Class: Big Data A
- Lecturer: Abdul Munif, S.Kom., M.Sc.

## Install & Initialization

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 25.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=128088b49b26b20b326737ed4d4a3c17d270e4989129595f269cda7cb86c550a
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [7]:
# SparkSession Initialization
spark = SparkSession.builder \
    .master("local") \
    .appName("MovieLens") \
    .getOrCreate()

In [8]:

# Read data from a text file and separate elements of each line
lines = spark.read.text("./sample_data/sample_movielens_ratings.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))

In [9]:
# Convert data into a DataFrame with userId, movieId, rating, and timestamp columns
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))

# Split the data into training (80%) and testing (20%) sets
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

## Build Recomendation model using ALS

In [11]:
# Initialize the parameters to be tried
max_iters = [5, 10, 20]
reg_params = [0.1, 0.5, 1.0]

# Dictionary to store RMSE results
results = {}

In [12]:
# Loop for every combination of maxIter and regParam
for max_iter in max_iters:
    for reg_param in reg_params:
        # Build the recommendation model using ALS on the training data
        # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        als = ALS(maxIter=max_iter, regParam=reg_param, userCol="userId", itemCol="movieId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(training)

        # Evaluate the model by computing the RMSE on the test data
        predictions = model.transform(test)
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")
        rmse = evaluator.evaluate(predictions)

        # Save the RMSE result in the dictionary
        results[(max_iter, reg_param)] = rmse
        print(f"Root-mean-square error for maxIter={max_iter}, regParam={reg_param} = {rmse}")

Root-mean-square error for maxIter=5, regParam=0.1 = 0.9962344961158077
Root-mean-square error for maxIter=5, regParam=0.5 = 1.2175132057715328
Root-mean-square error for maxIter=5, regParam=1.0 = 1.4992291118613619
Root-mean-square error for maxIter=10, regParam=0.1 = 0.9472275980351129
Root-mean-square error for maxIter=10, regParam=0.5 = 1.2176039367425477
Root-mean-square error for maxIter=10, regParam=1.0 = 1.4992329710998842
Root-mean-square error for maxIter=20, regParam=0.1 = 0.940150460366184
Root-mean-square error for maxIter=20, regParam=0.5 = 1.2178062084930956
Root-mean-square error for maxIter=20, regParam=1.0 = 1.4992329762155188


In [13]:
# Find the hyperparameter combination with the lowest RMSE
best_params = min(results, key=results.get)
best_rmse = results[best_params]
print(f"\nBest hyperparameters: maxIter={best_params[0]}, regParam={best_params[1]} with RMSE={best_rmse}")


Best hyperparameters: maxIter=20, regParam=0.1 with RMSE=0.940150460366184


## Generate Movie Recomendation

In [15]:
# Train the model with the best hyperparameters
best_als = ALS(maxIter=best_params[0], regParam=best_params[1], userCol="userId", itemCol="movieId", ratingCol="rating",
               coldStartStrategy="drop")
best_model = best_als.fit(training)

## Print Result and Show Ouput

In [16]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    20|[{94, 3.0157435},...|
|    10|[{2, 3.135369}, {...|
|     0|[{92, 3.0475888},...|
|     1|[{68, 3.1546354},...|
|    21|[{29, 4.3110056},...|
|    11|[{32, 4.7902164},...|
|    12|[{46, 4.5175467},...|
|    22|[{51, 4.492956}, ...|
|     2|[{93, 4.579589}, ...|
|    13|[{93, 3.0599294},...|
|     3|[{51, 4.1832976},...|
|    23|[{46, 4.92337}, {...|
|     4|[{29, 3.8232837},...|
|    24|[{69, 4.5547748},...|
|    14|[{52, 4.3371334},...|
|     5|[{46, 4.352494}, ...|
|    15|[{90, 3.2859805},...|
|    25|[{47, 2.8306553},...|
|    26|[{51, 4.763599}, ...|
|     6|[{25, 4.1410666},...|
+------+--------------------+
only showing top 20 rows



In [18]:
# Generate top 10 user recommendations for each movie
movieRecs = best_model.recommendForAllItems(10)
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     20|[{17, 3.9395316},...|
|     40|[{2, 3.556001}, {...|
|     10|[{23, 3.097157}, ...|
|     50|[{23, 3.7149057},...|
|     80|[{26, 3.3331735},...|
|     70|[{21, 3.2691026},...|
|     60|[{22, 2.78507}, {...|
|     90|[{17, 4.4264464},...|
|     30|[{22, 4.267375}, ...|
|      0|[{28, 2.3200734},...|
|     31|[{12, 2.677197}, ...|
|     81|[{28, 4.02462}, {...|
|     91|[{12, 2.945968}, ...|
|      1|[{25, 2.014217}, ...|
|     41|[{8, 3.2916598}, ...|
|     61|[{6, 2.1957097}, ...|
|     51|[{26, 4.763599}, ...|
|     21|[{26, 2.827591}, ...|
|     11|[{26, 1.1909164},...|
|     71|[{11, 2.646843}, ...|
+-------+--------------------+
only showing top 20 rows



In [19]:
# Generate top 10 movie recommendations for a specific set of users
users = ratings.select(best_als.getUserCol()).distinct().limit(3)
userSubsetRecs = best_model.recommendForUserSubset(users, 10)
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    26|[{51, 4.763599}, ...|
|    19|[{32, 2.9380393},...|
|    29|[{46, 4.1446085},...|
+------+--------------------+



In [20]:
# Generate top 10 user recommendations for a specific set of movies
movies = ratings.select(best_als.getItemCol()).distinct().limit(3)
movieSubSetRecs = best_model.recommendForItemSubset(movies, 10)
movieSubSetRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|     65|[{23, 4.3062}, {1...|
|     26|[{0, 2.302283}, {...|
|     29|[{8, 4.697743}, {...|
+-------+--------------------+



## Summary

The following sections describe the implemented steps:

* Data Loading and Preprocessing: </br>
We loaded the MovieLens dataset, processed it, and split it into training and test sets.

* Hyperparameter Tuning: </br>
We tried different combinations of maxIter and regParam values, evaluated the models on the test set, and stored the RMSE results in a dictionary.

* Model Selection: </br>
We identified the best hyperparameter combination based on the lowest RMSE and displayed the results.

* Generating Recommendations: </br>
We trained the model using the best hyperparameters and generated recommendations for all users, all movies, a subset of users, and a subset of movies.

The output shows the top 10 recommendations for each user, movie, and the specified subsets.